In [1]:
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor

import random
import re

In [3]:
# pipelines 
# The way pipelines works: 
# a yield dict from spider come here, if_else condition checks the desired key,
# and apply the cleaning functionn on it

class DataCleansingPipeline:
    def process_item(self, item, spider):
        if 'num_page' in item:
            item['num_page'] = self.clean_num_pages(''.join(item['num_page']))
        return item

    def clean_num_pages(self, text):
        match  = re.search('\d+\sPages', text)
        if match:
            cleaned_text =  match.group()
        else:
            cleaned_text = 'not mentioned'
        return cleaned_text

In [4]:
# This spider using the CrawlSpider class, do the scrapping of all pages of a popular category 
# and it extract the links of all books and parse it into parse_item for extraction
# another Rule is defined for pagnition

class PDFdriveCrawler(CrawlSpider):
    name='books-data-crawler'
    allowed_domains = ['pdfdrive.to']
    start_urls = ['https://pdfdrive.to/categories/popular']
    
    rule_books_link = Rule(LinkExtractor(restrict_css='a.title-link'), callback = 'parse_item', follow = False)
    rule_pagnition = Rule(LinkExtractor(restrict_xpaths="//span[@class='jsx-d375c0faf73020ef arrow-right']/.."), follow=True)
    rules = (rule_books_link, rule_pagnition, )
    

    def parse_item(self, response):
        yield{
        'book_name' : response.css('div h1::text').get(),
        'author_name' : response.xpath('//a[starts-with(@href, "/author/")]/text()').get(),
        'num_page' : response.css("div[class='jsx-7d4e960818f559df file-info'] span::text").getall(),
        'download_url' : response.url,
        'book_image' : response.css("div.container img[title='ebook img']::attr(src)").get(),
        }

In [ ]:
process = CrawlerProcess({
    
    'USER_AGENT' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.46',
    'FEED_FORMAT' : 'csv',
    'FEED_URI' : 'pdfdrive.csv',
    # 'DOWNLOAD_DELAY' : random.uniform(3.0, 5.0),
    'ITEM_PIPELINES': {'__main__.DataCleansingPipeline': 1}, # define for pipeline configuration
    'AUTOTHROTTLE_ENABLED' : True, #  This enable Auto Throttle extesnion, and it is pretty awesome.
    'AUTOTHROTTLE_START_DELAY' : 5.0,
    'AUTOTHROTTLE_MAX_DELAY' : 60.0,
    'AUTOTHROTTLE_TARGET_CONCURRENCY' :2.0
    
})

process.crawl(PDFdriveCrawler)
process.start()

2023-10-25 14:51:03 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2023-10-25 14:51:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.1.3 19 Sep 2023), cryptography 41.0.4, Platform Windows-10-10.0.19045-SP0
2023-10-25 14:51:03 [scrapy.addons] INFO: Enabled addons:
[]
2023-10-25 14:51:03 [py.warnings] WARNING: C:\Users\didx\AppData\Local\Programs\Python\Python311\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.